# Crawling the Web with julia

## Motivation

## Concrete example

For me currently, this is the Covid-19 pandemic. And more specifically, how is it developing close to me. In my case, that means the city of Hamburg in Germany.

There is the official website of the city, that has a page for the covid-19 numbers, (hamburg.de)[https://www.hamburg.de/corona-zahlen/].

The page with the numbers is in german, but don't worry, that's what our webcrawler will hopefully help us with - we can get to the numbers without reading all surrounding text. I will try to help out and translate what is relevant, but that will only be a minor detail when we try to find the right text to extract from.